In [ ]:
import pykx as kx

In [ ]:
vitals: `time xasc ("PIE";1#",")0:`:vitals.csv;

.pq:use`kx.pq;
t:.pq.pq`:weather.parquet;
weather: `time xasc select from t;

closest: aj[`time; vitals; weather];
\l pykx.q

In [ ]:
ans1: first[0!desc select count i by sensorId from closest]`sensorId;
ans2: count select from closest where heartRate > 100, temperatureF>77f;
ans3: first[0!desc select count i by hour:60 xbar time.minute from select from closest where heartRate > 100, temperatureF>77f]`hour;
show (ans1; ans2; ans3);

In [ ]:
%%py
import matplotlib.pyplot as plt
closest = kx.q("closest").pd()
closest['condition'] = (closest.heartRate>100) & (closest.temperatureF>77)
colors = closest['condition'].map({True: 'red', False: 'blue'})

_, ax = plt.subplots(1, 1)
closest.plot(
    x='heartRate', 
    y='temperatureF', 
    kind='scatter', 
    c=colors,
    ax=ax, 
    alpha=0.5, 
    title='Heart Rate vs. Ambient Temperature'
)

plt.show()